# Relationship between "frequency" and "phase" contd.

We saw how the simple "sine wave" can be represented using the mathematical function $y(t) = a \text{sin}(2\pi f t)$ where we said that $f$ is the "frequency" of the sine wave. Well, if you completed the last task in the first workshop notebook, you might've guessed that that was a lie ... sort of.

The part within the $\text{sin}(...)$ expression is called the **phase**, often denoted using the symbol $\phi$. As the value of the function evolves over time, the phase goes up linearly with time - since it is $\phi(t) = 2\pi{}ft$.

We need to understand the relationship between phase and frequency deeper if we want to understand how to deal with pure tones whose perceived frequency itself changes over time. If we naively try to substitute the constant $f$ with a function of $t$ like for example $f(t) = 400 if t < 5 else 800$, we can clearly see that we'll encounter a discontinuity in the phase at $t = 5$, which will cause a discontinuity in the value of the sine wave.

So, what's wrong with having a discontinuity in the audio signal? If you'd noticed either by listening or looking at the waveform, you'd have heard "clicks" whenever there is a discontinuity in the audio waveform. These "clicks" only get worse when we start considering more complicated frequency functions of time like what with music.

So what's the true relationship between "frequency" and "phase"? It is $2\pi f(t) = d\phi/dt$. You can see how this holds for our initial "since wave of fixed frequency" function. So a sine wave which has a time varying frequency ought to be determined using -

$$\text{sine_wave}(t) = a\text{sin}(\int_0^t{2\pi f(\tau)d\tau})$$

It is not as complicated to consider the case where the amplitude also varies with time --

$$\text{sine_wave}(t) = a(t) \text{sin}(\int_0^t{2\pi f(\tau)d\tau})$$


## Task

Can you try to express the other three waveforms in the first exercise -- the triangle wave, the square wave and the sawtooth wave -- in a similar manner?

## The "phasor"

The part of the sine wave expression $\int_0^t{f(\tau)d\tau}$ is a useful component to describe quasi periodic signals whose "frequency" keeps changing from time to time. We'll call this function the "phasor" - i.e. $\text{phasor}(t) = \int_0^t{f(\tau)d\tau}$. We can now express the sine wave (and other waves too) as $\text{sine_wave}(t) = a(t) \text{sin}(2\pi \text{phasor}(t))$. Note that the wave completes a period every time $\text{phasor}(t)$ crosses an integer.

## Rewriting our signal functions as stateful functions

If you noticed, we can no longer use the same approach to writing julia functions for these waves that we did at first. This is because to calculate the signal at any given time $t$, we will have to calculate an integral from the start of tiume $0$ to time $t$ ... which would be an expensive operation.

Since our `digitize` function calculates the signal value for incremental time steps, it would be adequate if we model our functions to also work in time increments of $T$ where $T$ is the time between two audio samples - i.e. "the sampling interval" -- where $f_s = 1/T$ gives the "sampling frequency" a.k.a. "sampling rate". Our mathematical function therefore has to now become "stateful" since we need to keep track of the "current phase" from one time step to the next.

In [22]:
# We will assume that amplitude and frequency are themselves time varying and therefore stateful.
# So now our "wave functions" are no longer functions of $t$ but are functions of $dt$ the time
# increment from one sample to the next.

# First, we define the ability to make a "constant waveform". This one doesn't change value from
# one time step to the next.
function konst(val)
    function wave(dt)
        val
    end
end

function phasor(frequency, initial_value = 0.0)
    p = initial_value
    function wave(dt)
        pval = p
        p = p + frequency(dt) * dt
        pval
    end
end

function sine_wave(amplitude, frequency, initial_phasor = 0.0)
    internal_phasor = phasor(frequency, initial_phasor)
    twopi = 2 * 3.141592654
    function wave(dt)
        amplitude(dt) * sin(twopi * internal_phasor(dt))
    end
end

# Our digitization function will now change to calling the functions with the 
# time increment instead of time.
function digitize(wavefn, duration, sampling_rate=48000)
    number_of_samples = trunc(Int, duration * sampling_rate)
    samples = Array{Float32,1}(undef, number_of_samples)
    dt = 1.0 / sampling_rate
    for i in 1:number_of_samples
        samples[i] = wavefn(dt)
    end
    samples
end

# We'll also write a utility function "scaled_wave" which will rescale the value of
# a time varying signal to a new range. If the given wavefn takes on values in the
# range [x0,x1], the scaled wave function will take on values in the range [y0,y1].
function scaled_wave(wavefn, x0, x1, y0, y1)
    scaling_factor = (y1 - y0) / (x1 - x0)
    function wave(dt)
        y0 + (wavefn(dt) - x0) * scaling_factor
    end
end

# Given samples and a file name, this one writes the float32 samples
# as raw data to the given output file.
function write_sound_file(file_name, samples, sampling_rate=48000)
    out = open(file_name, "w")
    write(out, samples)
    close(out)
end

write_sound_file (generic function with 2 methods)

We can now create a kind of siren-like sound that oscillates smoothly between two frequencies by making a sine wave whose frequency itself varyies as a sine wave. This is called "modulation".

In [21]:
amplitude = konst(0.25)
modulated_frequency = scaled_wave(sine_wave(konst(1.0), konst(0.5)), -1.0, 1.0, 440.0, 660.0)
siren = sine_wave(amplitude, modulated_frequency)
samples = digitize(siren, 10.0)
write_sound_file("/tmp/siren.float32", samples)

So you can now see how we can create and combine different wave types to make new kinds of sounds. Next up we'll explore how to analyze sounds and understand them from a perceptual point of view.

## Tasks

1. Write "stateful wave functions" like the above for the other waveforms you were introduced to - i.e. sawtooth wave, triangle wave, square wave and produce frequency modulated tones like we did with the sine wave. Tip - use the "phasor" approach.

2. Write a function (or many functions) that will sum up two or more "sine waves" of possibly different frequencies and amplitudes to produce a composite audio signal. Generate some examples, listen to them and observe any patterns in the way they sound.